<h1> Make json file </h1>

In [217]:
from __future__ import division
import gzip
import json
import numpy as np
import pandas as pd
from collections import OrderedDict
from os import listdir

In [36]:
# read weather data
weather_df = pd.read_csv('weather/addis_ababa__addis_ababa__addis_ketema__.csv', index_col=0)

In [49]:
# Read in all values as strings, or else the latitude, longitude precision gets truncated
woreda_df = pd.read_csv('/Users/attiladobi/zen_staging/elevation_area/woreda_info.csv', dtype=str)
#woreda_df.set_index('GeoKey', inplace=True)

# Read in all values as strings or else the latitude, longitude double precision gets truncated
#woreda_df = pd.read_csv('/Users/attiladobi/zen_staging/elevation_area/woreda_info.csv', dtype=str)

<h1> Use woreda_mapped file </h1>

In [165]:
def get_geo_code(name_list):
    region, zone, woreda = name_list
    return ('%s__%s__%s__' % (region, zone, woreda)).lower()

def load_woreda_mapped(woreda_mapped_path):
    # Load useing dtype=str to prevent truncating the double
    woreda_df = pd.read_csv(woreda_mapped_path, dtype=str)
    # Remove columns used for name matching
    column_cut = [column for column in woreda_df.columns if 'match' not in column]
    woreda_df = woreda_df[column_cut]
    # Add GeoKey column
    woreda_df['GeoKey'] = [get_geo_code(name_list) for name_list in woreda_df[['RegionName', 'ZoneName', 'WoredaName']].values]
    return woreda_df

In [220]:
woreda_mapped_path = '/Users/attiladobi/zenysis/pipeline/auto/ethiopia/bin/shared/data/woreda_mapped.csv'
HEADERS = ['Real_Date', 'RegionName', 'ZoneName', 'WoredaName', 'WoredaLat', 'WoredaLon', 'source', 'field', 'val']
FIELDS = ['precipitation', 'humid_max', 'humid_min', 'solar', 'temp_max', 'temp_min', 'wind_avg']

FOLDER_PATH = 'weather'

woreda_df = load_woreda_mapped(woreda_mapped_path)

with gzip.open('weather.json.gz', 'wr') as f:
    # loop through all of the Woreda weather data:
    for filename in [csvfile for csvfile in listdir(FOLDER_PATH) if '.csv' in csvfile]:
        # Load csv to dataframe
        weather_df = pd.read_csv('%s/%s' % (FOLDER_PATH, filename), index_col=0)
        # Inner join the two dataframes on the GeoKey
        weather_merge = pd.merge(weather_df, woreda_df, how='inner', on='GeoKey')
        
        for row in weather_merge.iterrows():
            row = row[1]
            # Avoid pandas truncating the double
            lat = float(row['WoredaLat'] )
            lon = float(row['WoredaLon'] )
            woreda_consts = row[['date', 'RegionName', 'ZoneName', 'WoredaName']].tolist() + [lat, lon, 'weather']
            # Loop throgh all of the fields. Skip nan values.
            for field, value in row[FIELDS].dropna().iteritems():
                val_list = woreda_consts + [field, value]
                # OderedDict to perserve header order
                f.write(json.dumps( OrderedDict([(key, value) for key, value in zip(HEADERS, val_list)])))
                f.write('\n')
